# Enhanced Question Answering Integrating Unstructured and Graph Knowledge using Neo4j and LangChain

In this notebook, we walk through the implementation of a sophisticated question-answering system, leveraging the synergistic capabilities of Neo4j and LangChain. The step-by-step guide emphasises the process of integrating unstructured data and graph knowledge, ensuring a comprehensive understanding of utilizing Neo4j Vector Index and GraphCypherQAChain for enhanced, informed response generation with Mistral-7b.

![semi-neo4j-langchain-pipeline.png](../../assets/img/semi-neo4j-langchain-pipeline.png)

In [6]:
!pip install langchain pydantic lxml
!pip install openai wikipedia tiktoken neo4j python-dotenv
!pip install -U sagemaker

In [12]:
!pip install "unstructured[all-docs]"

In [7]:
!pip install transformers
!pip install layoutparser torchvision

In [6]:
!pip install "detectron2@git+https://github.com/facebookresearch/detectron2"

  Cloning https://github.com/facebookresearch/detectron2 to /private/var/folders/1s/7ls98k5958g_xps8dp1jql080000gn/T/pip-install-2eex0u6z/detectron2_6b2121a5fafc4f108138adcc462a7a3c
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2 /private/var/folders/1s/7ls98k5958g_xps8dp1jql080000gn/T/pip-install-2eex0u6z/detectron2_6b2121a5fafc4f108138adcc462a7a3c
  Resolved https://github.com/facebookresearch/detectron2 to commit e9f7e2ba15abd7badcb05ef6f5076f06b36a9c5b
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp311-cp311-macosx_11_0_arm64.whl size=858172 sha256=5c75d977f66021bf61a240c2281a81aca46f41878565627c7f051e683294137f
  Stored in directory: /private/var/folders/1s/7ls98k5958g_xps8dp1jql080000gn/T/pip-ephem-wheel-cache-_z25ukzg/wheels/82/fd/8c/d3f45a18b1ac1b9e8c451858d5f394138eeb3a7c9f3bfc590b
Successfully built detectron2
  Attempting uninstall: detectron2
    Found existing installa

In [5]:
!conda install -c conda-forge poppler -y

Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.



In [4]:
!conda install -c conda-forge tesseract -y

Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.



Import important and necessary libraries to run the entire project

In [1]:
import requests
import getpass
import os
import re
import io
import base64
import numpy as np
from PIL import Image
from langchain.schema.messages import HumanMessage, SystemMessage
from operator import itemgetter
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from typing import Any
from pydantic import BaseModel
from unstructured.partition.auto import partition_pdf
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Neo4jVector
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
import uuid
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.schema.document import Document
from langchain.storage import InMemoryStore
from langchain.chat_models import ChatOpenAI, ChatOllama
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain_core.prompts.prompt import PromptTemplate
from dotenv import load_dotenv

In [2]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

Load the env variables securely from .env file

In [47]:
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["NEO4J_URI"] = os.getenv('NEO4J_URI_2')
os.environ["NEO4J_USERNAME"] = os.getenv('NEO4J_USERNAME')
os.environ["NEO4J_PASSWORD"] = os.getenv('NEO4J_PASSWORD')

url = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

llm_name = os.getenv('LLM')
ollama_base_url = os.getenv('OLLAMA_BASE_URL')


In [48]:
print(os.getenv('NEO4J_URI_2'))

bolt://localhost:7687


In [49]:
path = "pdf/"

Use unstructured library to partition pdf into text, table, and image data

In [50]:
raw_pdf_elements = partition_pdf(
    filename=path + "DIA_Dino_Celi_druck.pdf",
    # Using pdf format to find embedded image blocks
    # extract_images_in_pdf=True,
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    infer_table_structure=True,
    # Post processing to aggregate text once we have the title
    chunking_strategy="by_title",
    # Chunking params to aggregate text blocks
    # Attempt to create a new chunk 3800 chars
    # Attempt to keep chunks > 2000 chars
    # Hard max on chunks
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=path+'img',
)

In [12]:
class Element(BaseModel):
    type: str
    text: Any


# Categorize by type
categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]

## Neo4j Vector Index

Use the Parent-Child indexing and retrieval strategy

In [13]:
# Prompt
prompt_text = """You are an assistant tasked with summarizing tables and text. \
Give a concise summary of the table or text. Table or text chunk: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = ChatOllama(
            temperature=0,
            base_url=ollama_base_url,
            model=llm_name,
            #streaming=True,
            # seed=2,
            #top_k=10,  # A higher value (100) will give more diverse answers, while a lower value (10) will be more conservative.
            #top_p=0.3,  # Higher value (0.95) will lead to more diverse text, while a lower value (0.5) will generate more focused text.
            #num_ctx=3072,  # Sets the size of the context window used to generate the next token.
        )
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [14]:
# Apply to text
texts = [i.text for i in text_elements]
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 100})

In [15]:
# Apply to tables
tables = [i.text for i in table_elements]
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

In [9]:
import boto3

s3_client = boto3.client('s3', region_name='us-east-1')
bucket_name = 'neo4j-rag-img-s3'
for img_file in sorted(os.listdir('./img')):
    if img_file.endswith('.jpg'):
        img_path = os.path.join('./img', img_file)
        s3_client.upload_file(img_path, bucket_name, img_file)

In [10]:
def encode_image(image_path):
    ''' Getting the base64 string '''
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
def encode_image_from_uri(image_uri):
    ''' Getting the base64 string from an image URI '''
    response = requests.get(image_uri)
    if response.status_code == 200:
        return base64.b64encode(response.content).decode('utf-8')
    else:
        raise Exception(f"Failed to process image.")

def image_summarize(img_base64,prompt):
    ''' Image summary '''
    chat = ChatOpenAI(model="gpt-4-vision-preview",
                      max_tokens=1024)

    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text":prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{img_base64}"
                        },
                    },
                ]
            )
        ]
    )
    return msg.content

# Store base64 encoded images
img_base64_list = []
# Store image summaries
image_summaries = []
# Prompt
prompt = "Describe the image in detail. Be specific about graphs, such as bar plots."
# Read images, encode to base64 strings
response = s3_client.list_objects_v2(Bucket=bucket_name)
if 'Contents' in response:
    for item in response['Contents']:
        file_name = item['Key']
        if file_name.endswith('.jpg'):
            image_uri = f"https://{bucket_name}.s3.amazonaws.com/{file_name}"
            base64_image = encode_image_from_uri(image_uri)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image,prompt))

We will index the child chunks in a Neo4j vector index and store the parent documents in memory

In [38]:
# The vectorstore to use to index the child chunks
index_name = 'vector'

vectorstore = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(),
    url=os.environ["NEO4J_URI"], 
    username=os.environ["NEO4J_USERNAME"], 
    password=os.environ["NEO4J_PASSWORD"],
    index_name=index_name,
)

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

# Add image summaries
# img_ids = [str(uuid.uuid4()) for _ in img_base64_list]
# summary_img = [
#     Document(page_content=s, metadata={id_key: img_ids[i]})
#     for i, s in enumerate(image_summaries)
# ]
# retriever.vectorstore.add_documents(summary_img)
# retriever.docstore.mset(list(zip(img_ids, img_base64_list)))

TypeError: Neo4jVector.from_embeddings() missing 1 required positional argument: 'text_embeddings'

In [34]:
tables[0]

'2018 2019 2020 2021 Leistung (€ Mio.) 16.322,88 16.617,97 15.446,61 16.128,92 Umsatzerlöse (€ Mio.) 15.221,83 15.668,57 14.749,74 15.298,54 Auftragsbestand (€ Mio.) 16.899,71 17.411,48 18.369,02 22.500,85 Mitarbeiteranzahl (FTE) 75.460 76.919 74.340 73.606 ∆ % 10 11 6 0 2022 17.735,47 17.025,85 23.738,84 73.740'

In [35]:
table_summaries[0]

"The table shows the financial performance of a company from 2018 to 2022. Here is a summary of the key points:\n\n* Revenue (€ million): 15.221,83 in 2018, 15.668,57 in 2019, 14.749,74 in 2020, and 15.298,54 in 2021.\n* Earnings (€ million): 16.322,88 in 2018, 16.617,97 in 2019, 15.446,61 in 2020, and 16.128,92 in 2021.\n* Order backlog (€ million): 16.899,71 in 2018, 17.411,48 in 2019, 18.369,02 in 2020, and 22.500,85 in 2021.\n* Number of employees (full-time equivalents): 75.460 in 2018, 76.919 in 2019, 74.340 in 2020, and 73.606 in 2021.\n* Percentage change from the previous year: The company experienced a 10% increase in revenue in 2019, a 6% decrease in 2020, and a 0% increase in 2021.\n\nOverall, the company's financial performance improved from 2018 to 2019 but then declined in 2020 before recovering in 2021."

In [14]:
retriever.get_relevant_documents(
    "Which model outperforms on the Mmlu benchmark and is very similar to Palm-2?"
)[0]

'5.1. Text\n\n5.1.1. Academic Benchmarks\n\nWe compare Gemini Pro and Ultra to a suite of external LLMs and our previous best model PaLM 2 across a series of text-based academic benchmarks covering reasoning, reading comprehension, STEM, and coding. We report these results in Table 2. Broadly, we find that the performance of Gemini Pro outperforms inference-optimized models such as GPT-3.5 and performs comparably with several of the most capable models available, and Gemini Ultra outperforms all current models. In this section, we examine some of these findings.\n\nOn MMLU (Hendrycks et al., 2021a), Gemini Ultra can outperform all existing models, achieving an accuracy of 90.04%. MMLU is a holistic exam benchmark, which measures knowledge across a set of 57 subjects. Human expert performance is gauged at 89.8% by the benchmark authors, and Gemini Ultra is the first model to exceed this threshold, with the prior state-of-the-art result at 86.4%. Achieving high performance requires speci

In [15]:
from base64 import b64decode
def split_text_image(docs):
    b64 = []
    text = []
    for doc in docs:
        try:
            b64decode(doc)
            b64.append(doc)
        except Exception as e:
            text.append(doc)
    return {
        "images": b64,
        "texts": text
    }

def is_base64(s):
    try:
        if not re.match('^[A-Za-z0-9+/]+[=]{0,2}$', s):
            return False
        if len(s) % 4 != 0:
            return False
        base64.b64decode(s, validate=True)
        return True
    except Exception:
        return False

We will use the GPT-4V model for final generation as the retrieved information might contain image data

In [16]:
def gpt4v_prompt(dict):
    format_texts = "\n".join(dict["context"]["texts"])
    if len(dict['context']['images'])>0 and is_base64(dict['context']['images'][0]):
        content=[
            {"type": "text", "text": f"""Answer the question based only on the following context, which can include text, tables, and the below image:
                Question: {dict["question"]}

                Text and tables:
                {format_texts}
                """
            },
            {"type": "image_url", 
             "image_url": {"url": f"data:image/jpeg;base64,{dict['context']['images'][0]}"}},
        ]
    else:
        content=[
            {"type": "text", "text": f"""Answer the question based only on the following context, which can include text, tables, and the below image:
                Question: {dict["question"]}

                Text and tables:
                {format_texts}
                """
            }
        ]
        
    return [
            HumanMessage(
                content=content
            )
        ]
        

model = ChatOpenAI(temperature=0, model="gpt-4-vision-preview", max_tokens=1024)
chain = (
    {"context": retriever | RunnableLambda(split_text_image), "question": RunnablePassthrough()}
    | RunnableLambda(gpt4v_prompt)
    | model
    | StrOutputParser()
)

In [17]:
vector_result1 = chain.invoke(
    "Which model outperforms on the Mmlu benchmark and is very similar to Palm-2?"
)
vector_result1

'Gemini Ultra outperforms on the MMLU benchmark with an accuracy of 90.04%, which is very similar to Palm-2.'

In [18]:
vector_result2 = chain.invoke(
    "Which models apply Safety Filtering and Quality Filters"
)
vector_result2

'Safety Filtering and Quality Filters are applied to all Gemini models, as mentioned in the "Training Dataset" section of the provided text.'

## Neo4j DB QA chain

Once the graph is constructed, we need to connect to the Neo4j instance and print the schema.

In [19]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["NEO4J_URI"] = os.getenv('NEO4J_URI2')
os.environ["NEO4J_USERNAME"] = os.getenv('NEO4J_USERNAME')
os.environ["NEO4J_PASSWORD"] = os.getenv('NEO4J_PASSWORD2')

In [20]:
graph = Neo4jGraph(
    url=os.environ["NEO4J_URI"], 
    username=os.environ["NEO4J_USERNAME"], 
    password=os.environ["NEO4J_PASSWORD"],
)

In [21]:
print(graph.schema)

Node properties are the following:
Organization {name: STRING, id: STRING},Multimodal model {name: STRING, description: STRING, id: STRING},Team {name: STRING, affiliation: STRING, id: STRING},Model {name: STRING, version: STRING, id: STRING, sizes: STRING, performance: STRING, modelDescription: STRING, modelSize: STRING},Modelsize {id: STRING, name: STRING, description: STRING},Benchmark {description: STRING, id: STRING, name: STRING, accuracy: STRING},Person {id: STRING, name: STRING},Concept {id: STRING, name: STRING},Model family {id: STRING, name: STRING},Accelerator {id: STRING, name: STRING},Reference {name: STRING, id: STRING, year: STRING, authors: STRING},Challenge {name: STRING, description: STRING, id: STRING},Hardware {name: STRING, description: STRING, id: STRING},Network {description: STRING, id: STRING, references: STRING, name: STRING},Training paradigm {name: STRING, description: STRING, id: STRING},Programming model {description: STRING, id: STRING, name: STRING},Com

We are modifying the CYPHER_GENERATION_TEMPLATE and the CYPHER_QA_TEMPLATE template. This is because we have image uris associated with certain nodes in the graph and we aim to retrieve them and pass them to gpt4v for potentially better output response.

In [22]:
CYPHER_GENERATION_TEMPLATE = """Task: Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Only if the node associated with the result cypher query contains 'imageuri' property return it as well.

The question is:
{question}"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

In [23]:
CYPHER_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers.
The information part contains the provided information that you must use to construct an answer.
If the context contains link/imageuri, include it in the final answer output.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
If the provided information is empty, say that you don't know the answer.
Information:
{context}

Examples:
Question: What is the most fastest car?
Helpful Answer: The fastest car on the list is Bugatti Chiron. Image URI: https://manofmany.com/wp-content/uploads/2019/09/Bugatti-Chiron.jpg

Question: What is the most fastest car on Mars?
Helpful Answer: The fastest car on Mars is unknown. Image URI: None

Question: {question}
Helpful Answer:"""
CYPHER_QA_PROMPT = PromptTemplate(
    input_variables=["context", "question"], template=CYPHER_QA_TEMPLATE
)

In [24]:
graph_chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), 
    graph=graph, 
    cypher_prompt=CYPHER_GENERATION_PROMPT, 
    qa_prompt=CYPHER_QA_PROMPT,
    verbose=True
)

In [32]:
graph_result1 = graph_chain.run("Which model outperforms on the Mmlu benchmark and is very similar to Palm-2?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Model)-[:OUTPERFORMS]->(b:Benchmark {name: "Mmlu"})
MATCH (m)-[:COMPAREDTO]->(m2:Model {name: "Palm-2"})
RETURN m.name, m.imageuri
Full Context:
[{'m.name': 'Gemini Ultra', 'm.imageuri': None}]

> Finished chain.


In [33]:
graph_result1

'The model that outperforms on the Mmlu benchmark and is very similar to Palm-2 is Gemini Ultra. Image URI: None'

In [34]:
graph_result2 = graph_chain.run("Which models apply Safety Filtering and Quality Filters")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Model)-[:APPLY_FILTERS]->(f:Filter)
WHERE f.name = 'Safety Filtering' OR f.name = 'Quality Filters'
RETURN m.name, m.id, m.imageuri
Full Context:
[{'m.name': 'Gemini Models', 'm.id': 'Gemini Models', 'm.imageuri': None}, {'m.name': 'Gemini Models', 'm.id': 'Gemini Models', 'm.imageuri': None}]

> Finished chain.


In [35]:
graph_result2

'Gemini Models apply Safety Filtering and Quality Filters. Image URI: None'

We split the text and imageuri same as before because thats the format in which the gpt-4v model api requires us to pass text and image data.

In [36]:
def split_text_image(dict):
    res = dict['graph_result']
    http_index = res.find('http')

    if http_index != -1:
        image_url = res[http_index:].strip()
        text = res[:http_index].strip()

        return {
            "question": dict['question'],
            "image": image_url,
            "text": text.replace(" Image URI:", "")
        }
    else:
        return {
            "question": dict['question'],
            "image": None,
            "text": res.replace(" Image URI:", "")
        }

def is_base64(s):
    try:
        if not re.match('^[A-Za-z0-9+/]+[=]{0,2}$', s):
            return False
        if len(s) % 4 != 0:
            return False
        base64.b64decode(s, validate=True)
        return True
    except Exception:
        return False
    
def encode_image_from_uri(image_uri):
    ''' Getting the base64 string from an image URI '''
    response = requests.get(image_uri)
    if response.status_code == 200:
        return base64.b64encode(response.content).decode('utf-8')
    else:
        raise Exception(f"Failed to process image.")

In [37]:
def gpt4v_prompt(dict):
    if dict['image']!=None and is_base64(b64_img):
        b64_img = encode_image_from_uri(dict['image'])
        content=[
            {"type": "text", "text": f"""Answer the question based only on the following context, which can include text, tables, and the below image:
                Question: {dict["question"]}

                Text and tables:
                {dict['text']}
                """
            },
            {"type": "image_url", 
             "image_url": {"url": f"data:image/jpeg;base64,{b64_img}"}},
        ]
    else:
        content=[
            {"type": "text", "text": f"""Answer the question based only on the following context, which can include text, tables, and the below image:
                Question: {dict["question"]}

                Text and tables:
                {dict['text']}
                """
            }
        ]
        
    return [
            HumanMessage(
                content=content
            )
        ]
        

model = ChatOpenAI(temperature=0, model="gpt-4-vision-preview", max_tokens=1024)
chain = (
    {
        "question": itemgetter("question"),
        "graph_result": itemgetter("graph_result")
    }
    | RunnableLambda(split_text_image)
    | RunnableLambda(gpt4v_prompt)
    | model
    | StrOutputParser()
)

In [41]:
res = split_text_image({"graph_result": graph_result1, "question": "Which model outperforms on the Mmlu benchmark and is very similar to Palm-2?"})
if res["image"]!=None:
    graph_result1 = chain.invoke(
        {"question": "Which model outperforms on the Mmlu benchmark and is very similar to Palm-2?", "graph_result": graph_result1}
    )
else:
    graph_result1 = graph_result1.replace("Image URI: None", "")
graph_result1

'The model that outperforms on the Mmlu benchmark and is very similar to Palm-2 is Gemini Ultra. '

In [42]:
res = split_text_image({"graph_result": graph_result2, "question": "Which models apply Safety Filtering and Quality Filters"})
if res["image"]!=None:
    graph_result2 = chain.invoke(
        {"question": "Which models apply Safety Filtering and Quality Filters", "graph_result": graph_result2}
    )
else:
    graph_result2 = graph_result2.replace("Image URI: None", "")
graph_result2

'Gemini Models apply Safety Filtering and Quality Filters. '

## Mistral-7b-Instruct

We setup the Mistral-7B endpoint from Hugging Face within the AWS SageMaker environment.

In [43]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [44]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

hub = {
    'HF_MODEL_ID':'mistralai/Mistral-7B-Instruct-v0.1',
    'SM_NUM_GPUS': json.dumps(1)
}

In [45]:
huggingface_model = HuggingFaceModel(
    image_uri=get_huggingface_llm_image_uri("huggingface",version="1.1.0"),
    env=hub,
    role=role,
)

The final response is crafted by constructing a prompt that includes an instruction, relevant data from the vector index, relevant information from the graph database, and the user's query. This prompt is then passed to the Mistral-7b model, which generates a meaningful and accurate response based on the provided information.

In [46]:
mistral7b_predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.4xlarge",
    container_startup_health_check_timeout=300,
)

--------!

In [48]:
query = "Which model outperforms on the Mmlu benchmark and is very similar to Palm-2?"
final_prompt = f"""<s>[INST]You are a helpful question-answering agent. Use the below 
context to answer the question:

Context1: {vector_result1}
Context2: {graph_result1}

Question: {query}
Answer:[/INST]
"""

response = mistral7b_predictor.predict({
    "inputs": final_prompt,
})

print(re.search(r"Answer:\[\s*/INST\s*\]\n\n(.+)", response[0]['generated_text']).group(1))

Gemini Ultra outperforms on the MMLU benchmark and is very similar


In [49]:
query = "Which models apply Safety Filtering and Quality Filters"
final_prompt = f"""<s>[INST]You are a helpful question-answering agent. Use the below 
context to answer the question:

Context1: {vector_result2}
Context2: {graph_result2}

Question: {query}
Answer:[/INST]
"""

response = mistral7b_predictor.predict({
    "inputs": final_prompt,
})

print(re.search(r"Answer:\[\s*/INST\s*\]\n\n(.+)", response[0]['generated_text']).group(1))

Gemini models apply Safety Filtering and Quality Filters.
